In [ ]:
using PyCall
using PyPlot
using Statistics

yf = pyimport("yfinance")
np = pyimport("numpy")

include("OptimPortfolio.jl")
include("Utils.jl")

# Data donwload and basic analysis 

In [ ]:
start = "2013-01-01"
finish = "2017-01-01" 

start_test = "2017-01-01"
finish_test  ="2018-01-01"

assets = ["MSFT", "AAPL", "TSLA", "GOOG", "META", "AMZN"] #"^GSPC"

df = yf.download(assets, start, finish, progress=false)

df_test = yf.download(assets, start_test, finish_test, progress=false)

market = yf.download("^GSPC", start, finish, progress=false)

market_test = yf.download("^GSPC", start_test, finish_test, progress=false)

df_log_ret = (df["Adj Close"] / df["Adj Close"].shift(1)).apply(np.log)
df_test_log_ret = (df_test["Adj Close"] / df_test["Adj Close"].shift(1)).apply(np.log)

market_log_ret = (market["Adj Close"] / market["Adj Close"].shift(1)).apply(np.log)
market_test_log_ret = (market_test["Adj Close"] / market_test["Adj Close"].shift(1)).apply(np.log);

## Asset analysis

In [ ]:
fig, ax = plt.subplot_mosaic("""AB
    CD
    EF""", figsize=(8*2, 6*3))

# Stock prices
df["Adj Close"].plot(ax=ax["A"])

ax["A"].legend(ncol=2)

df_test["Adj Close"].plot(ax=ax["B"])

ax["B"].legend(ncol=2)

#Returns
df["Adj Close"].pct_change().plot(ax=ax["C"])
#df_log_ret.plot(ax=ax["C"])

ax["C"].legend(ncol=2)

df_test["Adj Close"].pct_change().plot(ax=ax["D"])
#df_test_log_ret.plot(ax=ax["D"])

ax["D"].legend(ncol=2)

#Histogram of returns
for asset in assets
    df["Adj Close"][asset].pct_change().hist(ax=ax["E"], bins=50, alpha=0.8, label=asset)
    #df_log_ret[asset].hist(ax=ax["E"], bins=50, alpha=0.8, label=asset)
end

ax["E"].legend(ncol=2)

for asset in assets
    df_test["Adj Close"][asset].pct_change().hist(ax=ax["F"], bins=30, alpha=0.8, label=asset)
    #df_test_log_ret[asset].hist(ax=ax["F"], bins=30, alpha=0.8, label=asset)
end

ax["F"].legend(ncol=2)

## Market analysis 

In [ ]:
fig, ax = plt.subplot_mosaic("""AB
    CD
    EF""", figsize=(8*2, 6*3))

# Stock prices
market["Adj Close"].plot(ax=ax["A"])

ax["A"].legend(ncol=2)

market_test["Adj Close"].plot(ax=ax["B"])

ax["B"].legend(ncol=2)

#Returns
market["Adj Close"].pct_change().plot(ax=ax["C"])
#market_log_ret.plot(ax=ax["C"])

ax["C"].legend(ncol=2)

market_test["Adj Close"].pct_change().plot(ax=ax["D"])
#market_test_log_ret.plot(ax=ax["D"])

ax["D"].legend(ncol=2)

#Histogram of returns
market["Adj Close"].pct_change().hist(ax=ax["E"], bins=50, alpha=0.8, label="S&P500")
#market_log_ret.hist(ax=ax["E"], bins=50, alpha=0.8, label="S&P500")

ax["E"].legend(ncol=2)

market_test["Adj Close"].pct_change().hist(ax=ax["F"], bins=20, alpha=0.8, label="S&P500")
#market_test_log_ret.hist(ax=ax["F"], bins=20, alpha=0.8, label="S&P500")

ax["F"].legend(ncol=2)

# Resample data to 1-month frequency

In [ ]:
freq = "1M"

df = df.resample(freq).mean()

df_test = df_test.resample(freq).mean()

market = market.resample(freq).mean()

market_test = market_test.resample(freq).mean()

df_log_ret = (df["Adj Close"] / df["Adj Close"].shift(1)).apply(np.log)
df_test_log_ret = (df_test["Adj Close"] / df_test["Adj Close"].shift(1)).apply(np.log)

market_log_ret = (market["Adj Close"] / market["Adj Close"].shift(1)).apply(np.log)
market_test_log_ret = (market_test["Adj Close"] / market_test["Adj Close"].shift(1)).apply(np.log);

# Mean-Variance portfolio optimization 

In [ ]:
returns = df["Adj Close"].pct_change().dropna().values
log_returns = df_log_ret.dropna().values

assets = [item for item in df["Adj Close"].columns]

portfolio = create_portfolio(returns, assets)
portfolio_log = create_portfolio(log_returns, assets);

### Fixed returns 

In [ ]:
target_return = 0.03

return_portfolio, risk_portfolio, w_opt = MV_fixed_return(portfolio, target_return; method="DCP")

return_market = market["Adj Close"].pct_change().mean()
risk_market = sqrt(market["Adj Close"].pct_change().var())

println("Expected Returns Portfolio: ", return_portfolio)
println("Risk Portfolio: ", risk_portfolio)
#println("Weights:", w_opt)

println("\nExpected Return Market: ", return_market)
println("Risk Market: ", risk_market)

println("\nPortfolio-market return ratio: ", return_portfolio/return_market)
println("Portfolio-market risk ratio: ", risk_portfolio/risk_market)

### Fixed risk 

In [ ]:
target_risk = 0.035

return_portfolio, risk_portfolio, w_opt = MV_fixed_risk(portfolio, target_risk; method="DCP")

return_market = market["Adj Close"].pct_change().mean()
risk_market = sqrt(market["Adj Close"].pct_change().var())

println("Expected Returns Portfolio: ", return_portfolio)
println("Risk Portfolio: ", risk_portfolio)
#println("Weights:", w_opt)

println("\nExpected Return Market: ", return_market)
println("Risk Market: ", risk_market)

println("\nPortfolio-market return ratio: ", return_portfolio/return_market)
println("Portfolio-market risk ratio: ", risk_portfolio/risk_market)

# Efficient frontier 

In [ ]:
RetRisk, weights = MV_efficient_frontier(portfolio, 100);

RetRisk_log, weights_log = MV_efficient_frontier(portfolio_log, 100);

risks = sqrt.(diag(portfolio.Σ))
risks_log = sqrt.(diag(portfolio_log.Σ))

plt.figure(figsize=(8*2, 6*2))

plt.subplot(2, 2, 1)

for i in 1:length(portfolio.μ)

    plt.scatter(risks[i], portfolio.μ[i], s=100, label=assets[i])
    
end

plt.scatter(risk_market, return_market, s=100, label="S&P500")

plt.plot(RetRisk[:, 2], RetRisk[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

plt.ylabel("Expected return", fontsize=20, labelpad=15)
plt.xlabel("Risk", fontsize=20, labelpad=15)

plt.legend(ncol=2)

plt.subplot(2, 2, 2)

for i in 1:length(portfolio.μ)

    plt.scatter(risks_log[i], portfolio_log.μ[i], s=100, label=assets[i])
    
end

plt.scatter(risk_market, return_market, s=100, label="S&P500")

plt.plot(RetRisk_log[:, 2], RetRisk_log[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

plt.ylabel("Expected log return", fontsize=20, labelpad=15)
plt.xlabel("Risk", fontsize=20, labelpad=15)

plt.legend(ncol=2)

#
plt.subplot(2,2,3)

N = Int(10^6)

rets = zeros(N)
risks = zeros(N)
sharpe_ratio = zeros(N)

for i in 1 : N

    w = rand(length(portfolio.μ))

    w = w / sum(w)
   
    return_portfolio = dot(w, portfolio.μ)
    risk_portfolio = sqrt(transpose(w)*portfolio.Σ*w)
    
    rets[i] = return_portfolio
    risks[i] = risk_portfolio
    
    sharpe_ratio[i] = return_portfolio / risk_portfolio
    
end

plt.plot(RetRisk[:, 2], RetRisk[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

plt.scatter(risks, rets, c=sharpe_ratio, s=8)

plt.ylabel("Expected return", fontsize=20, labelpad=15)
plt.xlabel("Risk", fontsize=20, labelpad=15)

plt.colorbar()

#
plt.subplot(2,2,4)

N = Int(10^6)

rets = zeros(N)
risks = zeros(N)
sharpe_ratio = zeros(N)

for i in 1 : N

    w = rand(length(portfolio_log.μ))

    w = w / sum(w)
   
    return_portfolio = dot(w, portfolio_log.μ)
    risk_portfolio = sqrt(transpose(w)*portfolio_log.Σ*w)
    
    rets[i] = return_portfolio
    risks[i] = risk_portfolio
    
    sharpe_ratio[i] = return_portfolio / risk_portfolio
    
end

plt.plot(RetRisk_log[:, 2], RetRisk_log[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

plt.scatter(risks, rets, c=sharpe_ratio, s=8)

plt.ylabel("Expected log return", fontsize=20, labelpad=15)
plt.xlabel("Risk", fontsize=20, labelpad=15)

plt.colorbar()

plt.subplots_adjust(wspace=0.3, hspace=0.3)

# Portfolio testing 

Up to this moment we have optimized the weights of our portfolio for any desired level of returns and risk. However, this is done in a training (in-sample) set, but in the future things could change and our optimal weights could'nt be optimal any more. Here we check how expected returns and expected risk for the optimized weights could vary in the future by applying our optimal weights to a test (out of sample) set. We will show 2 results:

- An example of the out-of-sample performance of the in-sample optimized portfolio yielding a fixed return of 0.025

- The Yield Curve of all in-sample optimal portfolios (the ones on the efficient frontier) in the out-of-sample set.

In [ ]:
#EXAMPLE

#Test in "future" data
df_test_returns = df_test["Adj Close"].pct_change().dropna().values

target_return = 0.025

#Optimize in past data
ret_opt, risk_opt, w_opt = MV_fixed_return(portfolio, target_return)

returns_test = df_test_returns * w_opt
    
risk = round(sqrt(var(returns_test)), digits=3)

dates = [df_test["Adj Close"].index[i] for i in 2: length(df_test)]

mean_return = round(mean(returns_test), digits=4)
return_market = round(market_test["Adj Close"].pct_change().mean(), digits=4)
risk_market = round(sqrt(market_test["Adj Close"].pct_change().var()), digits=3);

#YIELD CURVE
portfolio_test = create_portfolio(df_test_returns, assets) #Portfolio(df_test_returns.mean().values, df_test_returns.cov().values)

target_returns = 0.021 : 0.001 : 0.048

final_risks = []
final_returns = []

expected_risks = []
expected_returns = []

for target_return in target_returns

    expected_return, expected_risk, w_opt = MV_fixed_return(portfolio, target_return)

    returns = df_test_returns * w_opt

    actual_risk = round(sqrt(var(returns)), digits=6)
    actual_mean_return = round(mean(returns), digits=6)
    
    append!(expected_risks, expected_risk)
    append!(expected_returns, expected_return)
    
    append!(final_risks, actual_risk)
    append!(final_returns, actual_mean_return)
    
end

RetRisk_test, weights = MV_efficient_frontier(portfolio_test, 50);

In [ ]:
fig, ax = plt.subplot_mosaic("AB", figsize=(8*2,6))

ax["A"].plot(dates, returns_test, lw=3, marker="o", ms=12, label="Portfolio simulated returns")

ax["A"].axhline(mean_return, color="k", lw=3, label="Portfolio: μ=$mean_return,  σ=$risk")
ax["A"].axhline(return_market, color="k", lw=3, ls="--", label="Market: μ=$return_market,  σ=$risk_market")

ax["A"].tick_params("x", rotation=45)

ax["A"].legend()

ax["B"].scatter(risk_market, return_market, color="k", s=100, label="S&P500")

ax["B"].plot(expected_risks, expected_returns, marker="o", label="Expected Frontier", color="k")
ax["B"].plot(final_risks, final_returns, marker="o", label="Optimized Portfolios")

ax["B"].plot(RetRisk_test[:, 2], RetRisk_test[:, 1], color="C1", lw=1, ls="-", marker="o")

ax["B"].set_ylabel("Efficient return", fontsize=20, labelpad=15)
ax["B"].set_xlabel("Risk", fontsize=20, labelpad=15)

ax["B"].legend()

#INSET
axins = ax["B"].inset_axes([0.62, 0.1, 0.35, 0.4])

axins.plot(final_risks[1:8], final_returns[1:8], marker="o", color="r")
axins.plot(final_risks[9:end-12], final_returns[9:end-12], marker="o", color="g")
axins.plot(final_risks[end-12:end-7], final_returns[end-12:end-7], marker="o", color="r")

axins.plot(RetRisk_test[end-12:end, 2], RetRisk_test[end-12:end, 1], color="C1", lw=1, ls="-", marker="o")

axins.set_xticks([])
axins.set_yticks([])

ax["B"].indicate_inset_zoom(axins, edgecolor="black")

plt.subplots_adjust(wspace=0.3)

# Genetic Algorithm-based Portfolio Optimization

## Sanity check with MV model 

In [ ]:
#Efficient frontier using Convex Optimization
@time RetRisk_DCP, weights_DCP = MV_efficient_frontier(portfolio, 50; method="DCP")

#Efficient frontier using Evolutionary Optimization
@time RetRisk_EO, weights_EO = MV_efficient_frontier(portfolio, 50; method="EO")

plt.plot(RetRisk_DCP[:, 2], RetRisk_DCP[:, 1], color="g", lw=3, zorder=1, label="DCP")
plt.scatter(RetRisk_EO[:, 2], RetRisk_EO[:, 1], color="r", label="EO")

plt.legend()

## MVS model 

In [ ]:
#Minimize risk, subject to fixed return and skewness
function MVS_fixed_return_skewness(portfolio, target_return, target_skewness; w_lower=0.0, w_upper=1.0)

    f(w) = transpose(w)*portfolio.Σ*w

    ## 0<w_i<1
    lower = ones(6) .* w_lower
    upper = ones(6) .* w_upper

    ## sum(w) = 1
    c(w) = [sum(w), dot(w, portfolio.μ), skewness_portfolio(portfolio, w)]
    lc   = [1.0, target_return, target_skewness] # lower bound for constraint function
    uc   = [1.0, Inf, Inf]   # upper bound for constraint function

    con = PenaltyConstraints(1e8, lower, upper, lc, uc, c)

    #Define initial condition
    x0 = ones(length(portfolio.µ)) #./ length(portfolio.µ)

    #Optimize using Genetic Algorithm
    result = Evolutionary.optimize(f, con, x0, CMAES(μ=100, sigma0=0.01), 
                    Evolutionary.Options(iterations=Int(1e5), abstol=1e-12, reltol=1e-6))

    ret_opt = dot(result.minimizer, portfolio.μ)
    risk_opt = sqrt(transpose(result.minimizer)*portfolio.Σ*result.minimizer)
    skew_opt = skewness_portfolio(portfolio, result.minimizer)
    
    #Sanity Check
    if sum(result.minimizer) < 0.99

        println("Optimization did not converge...returning NaN.")

        return NaN, NaN, NaN, ones(length(portfolio.assets)) .* NaN

    else

        return ret_opt, risk_opt, skew_opt, result.minimizer

    end
    
end

#Maximize skewness subject to fixed return and risk
function MVS_fixed_return_risk(portfolio, target_return, target_risk; w_lower=0.0, w_upper=1.0)

    f(w) = -skewness_portfolio(portfolio, w)

    ## 0<w_i<1
    lower = ones(6) .* w_lower
    upper = ones(6) .* w_upper

    ## sum(w) = 1
    c(w) = [sum(w), dot(w, portfolio.μ), transpose(w)*portfolio.Σ*w]
    lc   = [1.0, target_return, 0.0] # lower bound for constraint function
    uc   = [1.0, Inf, target_risk^2]   # upper bound for constraint function

    con = PenaltyConstraints(1e8, lower, upper, lc, uc, c)

    #Define initial condition
    x0 = ones(length(portfolio.µ)) #./ length(portfolio.µ)

    #Optimize using Genetic Algorithm
    result = Evolutionary.optimize(f, con, x0, CMAES(μ=100, sigma0=0.01), 
                    Evolutionary.Options(iterations=Int(1e5), abstol=1e-12, reltol=1e-6))

    ret_opt = dot(result.minimizer, portfolio.μ)
    risk_opt = sqrt(transpose(result.minimizer)*portfolio.Σ*result.minimizer)
    skew_opt = skewness_portfolio(portfolio, result.minimizer)
    
    #Sanity Check
    if sum(result.minimizer) < 0.99

        println("Optimization did not converge...returning NaN.")

        return NaN, NaN, NaN, ones(length(portfolio.assets)) .* NaN

    else

        return ret_opt, risk_opt, skew_opt, result.minimizer

    end
    
end

#Maximize returns subject to fixed risk and skewness
function MVS_fixed_risk_skewness(portfolio, target_risk, target_skewness; w_lower=0.0, w_upper=1.0)

    f(w) = -dot(w, portfolio.μ)

    ## 0<w_i<1
    lower = ones(6) .* w_lower
    upper = ones(6) .* w_upper

    ## sum(w) = 1
    c(w) = [sum(w), transpose(w)*portfolio.Σ*w, skewness_portfolio(portfolio, w)]
    lc   = [1.0, 0.0, target_skewness] # lower bound for constraint function
    uc   = [1.0, target_risk^2, Inf]   # upper bound for constraint function

    con = PenaltyConstraints(1e8, lower, upper, lc, uc, c)

    #Define initial condition
    x0 = ones(length(portfolio.µ)) #./ length(portfolio.µ)

    #Optimize using Genetic Algorithm
    result = Evolutionary.optimize(f, con, x0, CMAES(μ=100, sigma0=0.01), 
                    Evolutionary.Options(iterations=Int(1e5), abstol=1e-12, reltol=1e-6))

    ret_opt = dot(result.minimizer, portfolio.μ)
    risk_opt = sqrt(transpose(result.minimizer)*portfolio.Σ*result.minimizer)
    skew_opt = skewness_portfolio(portfolio, result.minimizer)
    
    #Sanity Check
    if sum(result.minimizer) < 0.99

        println("Optimization did not converge...returning NaN.")

        return NaN, NaN, NaN, ones(length(portfolio.assets)) .* NaN

    else

        return ret_opt, risk_opt, skew_opt,  result.minimizer

    end
    
end

In [ ]:
target_return = 0.04

return_portfolio, risk_portfolio, w_opt = MV_fixed_return(portfolio, target_return; method="EO")

skew_p = skewness_portfolio(portfolio, w_opt)

println("Expected Returns Portfolio: ", return_portfolio)
println("Risk Portfolio: ", risk_portfolio)
println("Skewness Portfolio: ", skew_p)

In [ ]:
target_return = 0.04

target_risk = 0.08899

ret_opt, risk_opt, skew_opt, w_opt = MVS_fixed_return_risk(portfolio, target_return, target_risk)

println("Expected Returns Portfolio: ", ret_opt)
println("Risk Portfolio: ", risk_opt)
println("Skewness Portfolio: ", skew_opt)

### MVS efficient frontier

In MV optimization the efficient frontier can be constructed, for instance, by maximizing returns over different target risks. This process is independent on skewness. In the MVS model we construct the efficient frontier by maximizing skewness over the previously optimized values of returns and risk.

In [ ]:
target_returns = 0.026 : 0.0005 : 0.048 #Input

RetRiskSkew_MV = zeros(length(target_returns), 3)
weights_MV = zeros(length(target_returns), length(portfolio.μ))

RetRiskSkew_MVS = zeros(length(target_returns), 3)
weights_MVS = zeros(length(target_returns), length(portfolio.μ))

weight_differences = zeros(length(target_returns))

@time for i in 1 : length(target_returns)

    target_ret, target_risk, weight = MV_fixed_return(portfolio, target_returns[i]; method="DCP")
    
    skew_nonopt = skewness_portfolio(portfolio, weight)
    
    RetRiskSkew_MV[i, :] = [target_ret, target_risk, skew_nonopt]
    weights_MV[i, :] = weight
    
    ret_opt, risk_opt, skew_opt, w_opt = MVS_fixed_return_risk(portfolio, target_returns[i], target_risk)
    
    RetRiskSkew_MVS[i, :] = [ret_opt, risk_opt, skew_opt]
    weights_MVS[i, :] = w_opt
    
    weight_differences[i] = mean(abs.(weight .- w_opt))
    
end

In [ ]:
plt.figure(figsize=(8*2, 6))

plt.subplot(1, 2, 1)

plt.plot(target_returns, RetRiskSkew_MV[:, 3], lw=3, color="r", marker="o")
plt.plot(target_returns, RetRiskSkew_MVS[:, 3], lw=3, color="g", marker="o")

plt.subplot(1, 2, 2)
plt.plot(RetRiskSkew_MVS[:, 3] .- RetRiskSkew_MV[:, 3])

# TO DO 

- Sometime EO fail by not fulfiling sum(w)=1. Return NaN when this happens!

In [ ]:
points = 20

w_lower = 0.0
w_upper = 1.0

λ_1 = range(0.0, stop=1, length=points)
λ_2 = range(0.0, stop=1, length=points)

MeanVar = zeros((points, points, 4))

weights = zeros((points, points, length(portfolio.μ)))

## 0<w_i<1
lower = ones(6) .* w_lower
upper = ones(6) .* w_upper

## sum(w) = 1
c(w) = [sum(w)]
lc   = [1.0] # lower bound for constraint function
uc   = [1.0]   # upper bound for constraint function

con = PenaltyConstraints(1e8, lower, upper, lc, uc, c)

#Define initial condition
x0 = ones(length(portfolio.µ)) #./ length(portfolio.µ)

@time for i in 1:points
        
    for j in 1:points

        #Define objective function
        f(w) = λ_1[i] * (transpose(w)*portfolio.Σ*w)  - (1-λ_1[i]) * dot(w, portfolio.μ) - (1-λ_2[j]) * skewness_portfolio(portfolio, w) + λ_2[j] * kurtosis_portfolio(portfolio, w)

        #Optimize using Genetic Algorithm
        result = Evolutionary.optimize(f, con, x0, CMAES(μ=100, sigma0=0.01), 
                        Evolutionary.Options(iterations=Int(1e6), abstol=1e-12, reltol=1e-6))

        ret_opt = dot(result.minimizer, portfolio.μ)
        risk_opt = sqrt(transpose(result.minimizer)*portfolio.Σ*result.minimizer)
        skew_opt = skewness_portfolio(portfolio, result.minimizer)
        kurt_opt = kurtosis_portfolio(portfolio, result.minimizer)

        #Sanity Check
        if sum(result.minimizer) < 0.99

            #println("Optimization did not converge for λ=$(λs[i])...returning NaN.")

            MeanVar[i, j, :] = [NaN, NaN, NaN, NaN]
            weights[i, j, :] = ones(length(portfolio.assets)) .* NaN

        else

            MeanVar[i, j, :] = [ret_opt, risk_opt, skew_opt, kurt_opt]
            weights[i, j, :] = result.minimizer

        end	
        
    end
    
end

In [ ]:
fig, ax = plt.subplot_mosaic("""AB
    CD""", figsize=(8, 6))

ax["A"].scatter(MeanVar[:, :, 3], MeanVar[:,:, 4], c=MeanVar[:, :, 1], cmap="jet")

ax["B"].scatter(MeanVar[:, :, 3], MeanVar[:,:, 4], c=MeanVar[:, :, 2], cmap="jet")

ax["C"].scatter(MeanVar[:, :, 2],  MeanVar[:,:, 1], c=MeanVar[:,:, 3], cmap="jet")

ax["D"].scatter(MeanVar[:, :, 2],  MeanVar[:,:, 1], c=MeanVar[:,:, 4], cmap="jet")

# Higher moments 

In [ ]:
N = Int(10^6)

rets = zeros(N)
risks = zeros(N)
skews = zeros(N)
kurts = zeros(N)

sharpe_ratio = zeros(N)

for i in 1 : N

    w = rand(length(portfolio_log.μ))

    w = w / sum(w)
   
    return_portfolio = dot(w, portfolio.μ)
    risk_portfolio = sqrt(transpose(w)*portfolio.Σ*w)
    skew_portfolio = skewness_portfolio(portfolio, w)
    kurt_portfolio = kurtosis_portfolio(portfolio, w)
    
    rets[i] = return_portfolio
    risks[i] = risk_portfolio
    skews[i] = skew_portfolio
    kurts[i] = kurt_portfolio
    
    sharpe_ratio[i] = return_portfolio / risk_portfolio
    
end

In [ ]:
fig, ax = plt.subplot_mosaic("""AB
    CD""", figsize=(8*2, 6*2))


ax["A"].scatter(skews, kurts, c=rets, cmap="jet")

ax["B"].scatter(skews, kurts, c=risks, cmap="jet")

ax["C"].scatter(risks, rets, c=skews, cmap="jet")
ax["C"].plot(RetRisk_DCP[:, 2], RetRisk_DCP[:, 1], color="k", lw=3, label="Efficient frontier")

ax["D"].scatter(risks, rets, c=kurts, cmap="jet")
ax["D"].plot(RetRisk_DCP[:, 2], RetRisk_DCP[:, 1], color="k", lw=3, label="Efficient frontier")
